In [1]:
import sys, logging
from pathlib import Path
import numpy as np

# Agregar src al path (ajusta si el notebook está en otro directorio)
sys.path.insert(0, str(Path.cwd() / "src"))

from patches_tda import PatchSpacePipeline, PipelineConfig, WitnessTDA,PolynomialPatchGenerator
from patches_tda.ui import build_ui, PatchGeneratorAdapter


In [ ]:
DATA_DIR = Path("vanhateren_imc")

PATCH_SIZE = 3
PATCHES_PER_IMAGE = 5000
TOP_DNORM_FRACTION = 0.20

TARGET_PATCH_SPACE_SIZE = 1_000_000
SUBSAMPLE_SIZE = 50_000

DENSITY_K = 15
DENSITY_TOP_FRACTION = 0.30
DENOISE_ITERATIONS = 2

SEED = 42
MAX_IMAGES = 500  # None = todas

# salida: ahora mejor como .npk (tu decisión)
OUTPUT_FILE = Path(f"X({DENSITY_K}, {DENSITY_TOP_FRACTION}).npk")


In [ ]:
logging.basicConfig(level=logging.INFO, format="%(levelname)s | %(name)s | %(message)s")
logger = logging.getLogger("notebook")

In [ ]:
if not DATA_DIR.exists():
    raise FileNotFoundError(f"No existe: {DATA_DIR}")

config = PipelineConfig(
    data_dir=DATA_DIR,
    patch_size=PATCH_SIZE,
    patches_per_image=PATCHES_PER_IMAGE,
    top_dnorm_fraction=TOP_DNORM_FRACTION,
    target_patch_space_size=TARGET_PATCH_SPACE_SIZE,
    subsample_size=SUBSAMPLE_SIZE,
    density_k=DENSITY_K,
    density_top_fraction=DENSITY_TOP_FRACTION,
    denoise_iterations=DENOISE_ITERATIONS,
    seed=SEED,
    max_images=MAX_IMAGES,
)

pipeline = PatchSpacePipeline(config)
X_pk = pipeline.run()
stats = pipeline.last_stats

X_pk.shape, X_pk.dtype

In [ ]:
print("RESULTADOS PATCH SPACE")
print("  imágenes procesadas:", stats.images_processed)
print("  parches extraídos:", f"{stats.total_patches_extracted:,}")
print("  tras D-norm:", f"{stats.patches_after_dnorm:,}")
print("  patch space size:", f"{stats.patch_space_size:,}")
print("  subsample:", f"{stats.subsample_size:,}")
print("  X(p,k) final:", f"{stats.final_size:,}")
print("  shape:", X_pk.shape)
print("  memoria MB:", X_pk.nbytes/(1024*1024))

In [ ]:
# Nota: np.save siempre añade .npy si no está, así que para .npk usa esto:
with open(OUTPUT_FILE, "wb") as f:
    np.save(f, X_pk)

print("Guardado en:", OUTPUT_FILE)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.load(OUTPUT_FILE)   # <-- ESTA LÍNEA FALTABA

plt.figure(figsize=(6, 6))
plt.scatter(X[:, 0], X[:,1], s=2, alpha=0.6)
plt.xlabel("x")
plt.ylabel("y")
plt.gca().set_aspect("equal", adjustable="box")
plt.tight_layout()
plt.show()

In [ ]:
TDA_M_POINTS = 2000
TDA_N_LANDMARKS = 100
TDA_N_WITNESSES = 1000
TDA_MAX_ALPHA_SQUARE = 0.5
TDA_LIMIT_DIMENSION = 2

In [ ]:
tda = (
    WitnessTDA(
        data_path=OUTPUT_FILE,
        m_points=TDA_M_POINTS,
        n_landmarks=TDA_N_LANDMARKS,
        n_witnesses=TDA_N_WITNESSES,
        max_alpha_square=TDA_MAX_ALPHA_SQUARE,
        limit_dimension=TDA_LIMIT_DIMENSION,
        seed=SEED
    )
    .load()
    .sample()
    .compute()
)

print("Puntos usados:", tda.points_.shape[0])
print("Landmarks:", tda.landmarks_.shape[0])
print("Witnesses:", tda.witnesses_.shape[0])
print("Intervalos:", len(tda.persistence_))

In [ ]:
tda.plot_projection(method="coords", coords=(0,1))


# o si quieres coordenadas:
# tda.plot_projection(method="coords", dims=(0,1))

In [ ]:
# Si tu WitnessTDA ya trae plot_persistence() interno:
tda.plot_persistence()

# o si usa función externa manual:
# tda.plot_persistence(plot_persistence_manual, title_prefix="Witness", max_dims=(0,1,2))


In [ ]:
tda.plot_barcodes(max_dims=(0,1,))


In [ ]:
gen = PolynomialPatchGenerator(patch_size=3, usar_norma_D =False)
patch = gen.generate_patch(theta=np.pi/4, phi=np.pi/3)
gen.plot_patch(patch, title="Mi parche")

# Grilla de variaciones
gen.plot_patch_grid(n_theta=4, n_phi=4)

In [2]:
%matplotlib widget

In [3]:

gen = PolynomialPatchGenerator(patch_size=3, usar_norma_D =False)
# Crear adapter (traduce generate_patch → generate)
adapter = PatchGeneratorAdapter(gen)
# Construir UI
ui = build_ui(adapter)
# Mostrar
display(ui)